In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# FOR PHASE 1 of Training
def lr_schedule_1(epoch): 
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

# FOR PHASE 2 of Training
def lr_schedule_2(epoch):  
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    elif epoch > 45:
        lrate = 0.0003        
    return lrate

# FOR PHASE 3 of Training
def lr_schedule_3(epoch): 
    lrate = 0.001
    #if epoch > 15:
    #    lrate = 0.0005
    #elif epoch > 25:
    #    lrate = 0.0003        
    return lrate
    
#Data normalization as per CIFAR-10 dataset instructions - https://www.cs.toronto.edu/~kriz/cifar.html
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)



In [ ]:
# Construct CNN

weight_decay = 1e-4
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())


model.add(Flatten())
# Add softmax (output) layer
model.add(Dense(num_classes, activation='softmax'))

#data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Only one model.fit call of the 3 listed below must be uncommented while execution

# For Phase 1: 125 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_1)])

# For Phase 2: 60 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=60,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_2)])

# For Phase 3: 30 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=30,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

In [ ]:
y_pred_test = model.predict(x_test)
#y_pred_test
joint_prob = np.zeros((10,10)) #for CIFAR 10 (i,j) - state (true label) i, action(prediction prob) j
for i in range(10):
  idx  = np.where(np.argmax(y_test,1)==i)
  for j in range(10):
    joint_prob[i,j] = np.mean(y_pred_test[idx,j])

joint_prob = joint_prob*0.1 # prior pmf over the set of true image classes in CIFAR-10  is uniform.

# joint_prob[x,a] is the joint probability that the true image class is 'x' and predicted image class is 'a'
joint_prob

array([[6.52847409e-01, 3.89772281e-02, 6.80179894e-02, 7.88423419e-03,
        2.67848391e-02, 1.47459528e-03, 2.24223509e-02, 1.40603101e-02,
        1.25923946e-01, 4.16071080e-02],
       [2.07927134e-02, 8.35926354e-01, 5.51893841e-03, 1.05584657e-03,
        1.65963336e-03, 4.85769182e-04, 1.34579334e-02, 2.58387555e-03,
        4.10170369e-02, 7.75019377e-02],
       [7.00634792e-02, 1.00598987e-02, 5.40355265e-01, 2.67210137e-02,
        9.42950323e-02, 2.17632744e-02, 1.49639159e-01, 4.48053740e-02,
        2.97557190e-02, 1.25417672e-02],
       [3.56115177e-02, 2.25832909e-02, 8.77131447e-02, 3.01845223e-01,
        1.04545139e-01, 8.76664147e-02, 2.38369644e-01, 5.15195094e-02,
        5.08232526e-02, 1.93228573e-02],
       [2.27273759e-02, 2.54073204e-03, 9.02290493e-02, 2.34172698e-02,
        6.13664269e-01, 1.38230436e-02, 1.56577125e-01, 5.10738455e-02,
        2.11137049e-02, 4.83361445e-03],
       [1.66481696e-02, 8.20870418e-03, 8.46087039e-02, 1.34385675e-01,
   

In [ ]:
#Code structure credits :  https://github.com/abhijeet3922/Object-recognition-CIFAR-10/blob/master/cifar10_90.py